In [2]:
from main.compiling.compilers.AncillaPerCheckCompiler import AncillaPerCheckCompiler
from main.compiling.noise.models.CircuitLevelNoise import CircuitLevelNoise
from main.compiling.syndrome_extraction.extractors.ancilla_per_check.mixed.CxCyCzExtractor import CxCyCzExtractor
from main.codes.tic_tac_toe.HoneycombCode import HoneycombCode
from main.codes.tic_tac_toe.gauge_honeycomb_code import GaugeHoneycombCode
from main.codes.tic_tac_toe.gauge_floquet_colour_code import GaugeFloquetColourCode
from main.codes.tic_tac_toe.FloquetColourCode import FloquetColourCode
from main.codes.tic_tac_toe.TicTacToeCode import TicTacToeCode
from main.utils.enums import State
from main.building_blocks.pauli import Pauli
from main.building_blocks.pauli.PauliLetter import PauliLetter
import stim
import sinter 
import matplotlib.pyplot as plt
import numpy as np
from beliefmatching import BeliefMatchingSinterDecoder
from main.utils.Colour import Blue, Green, Red

from main.building_blocks.detectors.Stabilizer import Stabilizer


class HoneycombCodeZ(TicTacToeCode):
    """Adaptation of the Honeycomb code where the measurement order has been permuted
    """
    def __init__(self, distance: int):
        tic_tac_toe = [
                (Blue, PauliLetter('Z')),
                (Red, PauliLetter('X')),
                (Green, PauliLetter('Y'))]

        super().__init__(distance, tic_tac_toe)

# load the data.
# seperate running and loading.



# Help functions

In [3]:
def calculate_threshold(code_name, distances, observable_index, layer_factor,pers):
    tasks = []
    if code_name == "HoneycombCodeX":
        code = HoneycombCode
    elif code_name == "HoneycombCodeZ":
        code = HoneycombCodeZ

    elif code_name == "GaugeHoneycombCodeX":
        code = GaugeHoneycombCode
        gauge_factor = 2
    elif code_name == "FloquetColourCodeX":
        code = FloquetColourCode

    elif code_name == "GaugeFloquetColourCodeX":
        code = GaugeFloquetColourCode
        gauge_factor = 2



    for d in distances:
        for index, p in enumerate(pers):

            # generate circuit
            if code_name[-1] == 'X':
                if code_name == "GaugeHoneycombCodeX" or code_name == "GaugeFloquetColourCodeX":
                    circuit = get_X_stim_circuit(code(d,gauge_factor), observable_index,layer_factor*d, p)
                else:
                    circuit = get_X_stim_circuit(code(d), observable_index,layer_factor*d, p)
            elif code_name[-1] == 'Z':
                circuit = get_Z_stim_circuit(code(d, gauge_factor), observable_index, layer_factor*d, p)
            tasks.append(sinter.Task(
                    circuit=circuit,
                    detector_error_model=circuit.detector_error_model(decompose_errors=True, approximate_disjoint_errors=True, ignore_decomposition_failures=True),
                    json_metadata={
                        'code': code_name,
                        'distance': d,
                        'p': p,
                        'layers': layer_factor*d,
                        'r': layer_factor}))

                        

    samples = sinter.collect(
        tasks=tasks,
        hint_num_tasks=len(tasks),
        num_workers=3,
        max_shots=100000,
        max_errors=1000,
        decoders=['pymatching'],#,'beliefmatching'],
        custom_decoders={'beliefmatching': BeliefMatchingSinterDecoder()},
        print_progress=True,
        save_resume_filepath=f'./resume_11_2/{code_name}_{observable_index}_{layer_factor}.json',)
    return(samples)


def get_X_stim_circuit(code,observable_index,layers,p):
    compiler = AncillaPerCheckCompiler(CircuitLevelNoise(p,p,p,p,p), syndrome_extractor=CxCyCzExtractor())
    data_qubits = code.data_qubits.values()
    data_qubit_initial_states = {qubit: State.Plus for qubit in data_qubits}
    final_measurements = [Pauli(qubit, PauliLetter('X')) for qubit in data_qubits]
    logical_observables = [code.logical_qubits[observable_index].x]#, code.logical_qubits[1].x]

    initial_stabilizers = []
    for check in code.check_schedule[0]:
        initial_stabilizers.append(Stabilizer([(0,check)],0))
    stim_circuit: stim.Circuit = compiler.compile_to_stim(code,layers=layers, initial_stabilizers=initial_stabilizers,final_measurements=final_measurements,logical_observables=logical_observables)
    return(stim_circuit)

def get_Z_stim_circuit(code,observable_index,layers,p):
    compiler = AncillaPerCheckCompiler(noise_model=CircuitLevelNoise(p,p,p,p,p), syndrome_extractor=CxCyCzExtractor())
    data_qubits = code.data_qubits.values()
    data_qubit_initial_states = {qubit: State.Zero for qubit in data_qubits}
    final_measurements = [Pauli(qubit, PauliLetter('Z')) for qubit in data_qubits]
    logical_observables = [code.logical_qubits[observable_index].z]
    stim_circuit: stim.Circuit = compiler.compile_to_stim(code,layers=layers,initial_states= data_qubit_initial_states, final_measurements=final_measurements,logical_observables=logical_observables)
    return(stim_circuit)

In [3]:
#hcc_circuit = get_X_stim_circuit(HoneycombCode(4), 0,48,0.1)
#fcc_circuit = get_X_stim_circuit(FloquetColourCode(4), 0,24,0.1)
#print(hcc_circuit.num_detectors)
#print(fcc_circuit.num_detectors)

hcc_circuit = get_X_stim_circuit(HoneycombCode(4), 0,8,0.1)
print(hcc_circuit.detector_error_model(approximate_disjoint_errors=True).num_detectors)

fcc_circuit = get_X_stim_circuit(FloquetColourCode(4), 0,4,0.1)
print(fcc_circuit.detector_error_model(approximate_disjoint_errors=True).num_detectors)

gauge_hcc_circuit = get_X_stim_circuit(GaugeHoneycombCode(4,2),0, 2, 0.1)
print(gauge_hcc_circuit.detector_error_model(approximate_disjoint_errors=True).num_detectors)


gauge_fcc_circuit = get_X_stim_circuit(GaugeFloquetColourCode(4,2),0, 1, 0.1)
gauge_fcc_circuit.detector_error_model(approximate_disjoint_errors=True,decompose_errors=True).diagram("matchgraph-3d")


|████████████████████████████████████████✗︎ (!) 195/171 [114%] in 0.1s (1404.40/s) 
104
|████████████████████████████████████████✗︎ (!) 195/171 [114%] in 0.2s (1048.59/s) 
104
|████████████████████████████████████████✗︎ (!) 195/171 [114%] in 0.2s (923.19/s)                                        
200
|████████████████████████████████████████✗︎ (!) 195/171 [114%] in 0.4s (547.93/s)                                        


{"accessors":[{"bufferView":0,"byteOffset":0,"componentType":5126,"count":17,"max":[0,0.400000005960464,0.400000005960464],"min":[0,-0.400000005960464,-0.400000005960464],"name":"circle_loop","type":"VEC3"},{"bufferView":1,"byteOffset":0,"componentType":5126,"count":4376,"max":[75.5652389526367,40.2531852722168,78.8190765380859],"min":[-6.02027225494385,-4.79622793197632,-0.849162101745605],"name":"buf_scattered_lines","type":"VEC3"},{"bufferView":2,"byteOffset":0,"componentType":5126,"count":368,"max":[66,33,72],"min":[3,0,0],"name":"buf_red_scattered_lines","type":"VEC3"}],"asset":{"version":"2.0"},"bufferViews":[{"buffer":0,"byteLength":204,"byteOffset":0,"name":"circle_loop","target":34962},{"buffer":1,"byteLength":52512,"byteOffset":0,"name":"buf_scattered_lines","target":34962},{"buffer":2,"byteLength":4416,"byteOffset":0,"name":"buf_red_scattered_lines","target":34962}],"buffers":[{"byteLength":204,"name":"circle_loop","uri":"data:application/octet-stream;base64,AAAAAM3MzD4AAAAAAAAAAOU1vT5Fvxw+AAAAAMPQkD7D0JA+AAAAAES/HD7lNb0+AAAAAPIwlrLNzMw+AAAAAEe/HL7lNb0+AAAAAMPQkL7D0JA+AAAAAOc1vb5Avxw+AAAAAM3MzL7yMBazAAAAAOU1vb5Evxy+AAAAAMHQkL7E0JC+AAAAADy/HL7nNb2+AAAAAPLkozHNzMy+AAAAAEm/HD7kNb2+AAAAAMbQkD6/0JC+AAAAAOY1vT5Evxy+AAAAAM3MzD4AAAAA"},{"byteLength":52512,"name":"buf_scattered_lines","uri":"data:application/octet-stream;base64,AACoQQAABEIAAAAAAACoQQAABEIAAEBAAACoQQAABEIAAAAAAADwQQAAwEEAAEBBAACoQQAABEIAAAAAAABAQQAAwEAAAEBBAABAQAAABEIAAAAAAABAQAAABEIAAEBAAABAQAAABEIAAAAAAABAQQAAwEAAAEBBAABAQAAABEIAAAAAAACEQgAAwEEAAEBBAABAQQAAwEEAAAAAAABAQQAAwEEAAEBAAABAQQAAwEEAAAAAAADwQQAAwEEAAEBBAABAQQAAwEEAAAAAAACEQgAAwEEAAEBBAAAcQgAAcEEAAAAAAAAcQgAAcEEAAEBAAAAcQgAAcEEAAAAAAABAQgAAwEAAAEBBAACoQQAAcEEAAAAAAACoQQAAcEEAAEBAAACoQQAAcEEAAAAAAABAQQAAwEAAAEBBAADwQQAAwEAAAAAAAADwQQAAwEAAAEBAAADwQQAAwEAAAAAAAABAQQAAwEAAAEBBAADwQQAAwEAAAAAAAABAQgAAwEAAAEBBAABkQgAABEIAAAAAAABkQgAABEIAAEBAAABkQgAABEIAAAAAAABAQgAAwEAAAEBBAABkQgAABEIAAAAAAACEQgAAwEEAAEBBAAAcQgAABEIAAAAAAAAcQgAABEIAAEBAAAAcQgAABEIAAAAAAADwQQAAwEEAAEBBAAAcQgAABEIAAAAAAABAQgAAwEAAAEBBAABAQgAAwEEAAAAAAABAQgAAwEEAAEBAAABAQgAAwEEAAAAAAADwQQAAwEEAAEBBAABAQgAAwEEAAAAAAACEQgAAwEEAAEBBAABAQAAAcEEAAAAAAABAQAAAcEEAAEBAAABAQAAAcEEAAAAAAABAQQAAwEAAAEBBAABkQgAAcEEAAAAAAABkQgAAcEEAAEBAAABkQgAAcEEAAAAAAABAQgAAwEAAAEBBAACEQgAAwEAAAAAAAACEQgAAwEAAAEBAAACEQgAAwEAAAAAAAABAQQAAwEAAAEBBAACEQgAAwEAAAAAAAABAQgAAwEAAAEBBAACoQQAABEIAAEBAAADwQQAAwEEAAEBBAACoQQAABEIAAEBAAABAQQAAwEAAAEBBAACoQQAABEIAAEBAAABAQQAA8EEAAMBBAABAQAAABEIAAEBAAABAQQAAwEAAAEBBAABAQAAABEIAAEBAAACEQgAAwEEAAEBBAABAQAAABEIAAEBAAABAQQAA8EEAAMBBAABAQQAAwEEAAEBAAADwQQAAwEEAAEBBAABAQQAAwEEAAEBAAACEQgAAwEEAAEBBAABAQQAAwEEAAEBAAABAQQAA8EEAAMBBAAAcQgAAcEEAAEBAAABAQgAAwEAAAEBBAAAcQgAAcEEAAEBAAADwQQAAQEEAAMBBAACoQQAAcEEAAEBAAABAQQAAwEAAAEBBAACoQQAAcEEAAEBAAADwQQAAQEEAAMBBAADwQQAAwEAAAEBAAABAQQAAwEAAAEBBAADwQQAAwEAAAEBAAABAQgAAwEAAAEBBAADwQQAAwEAAAEBAAADwQQAAQEEAAMBBAABkQgAABEIAAEBAAABAQgAAwEAAAEBBAABkQgAABEIAAEBAAACEQgAAwEEAAEBBAABkQgAABEIAAEBAAABAQgAA8EEAAMBBAAAcQgAABEIAAEBAAADwQQAAwEEAAEBBAAAcQgAABEIAAEBAAABAQgAAwEAAAEBBAAAcQgAABEIAAEBAAABAQgAA8EEAAMBBAABAQgAAwEEAAEBAAADwQQAAwEEAAEBBAABAQgAAwEEAAEBAAACEQgAAwEEAAEBBAABAQgAAwEEAAEBAAABAQgAA8EEAAMBBAABAQAAAcEEAAEBAAABAQQAAwEAAAEBBAABAQAAAcEEAAEBAAACEQgAAQEEAAMBBAABkQgAAcEEAAEBAAABAQgAAwEAAAEBBAABkQgAAcEEAAEBAAACEQgAAQEEAAMBBAACEQgAAwEAAAEBAAABAQQAAwEAAAEBBAACEQgAAwEAAAEBAAABAQgAAwEAAAEBBAACEQgAAwEAAAEBAAACEQgAAQEEAAMBBAACoQQAAEEEAABBBTUeFQVbX0kBgpqM+AACoQQAAEEEAABBBAABAQQAAwEAAAPBBAABAQAAAEEEAABBBPhaPwBYT50AEIyZAAABAQAAAEEEAABBBAABAQQAAwEAAAPBBAABAQQAAAAAAABBBUI/GQE79h8CO8QJAAABAQQAAAAAAABBBAABAQQAAwEAAAPBBAAAcQgAA2EEAABBBxiMiQvCm9EHAoVa+AAAcQgAA2EEAABBBAADwQQAAwEEAAPBBAACoQQAA2EEAABBBbzWGQThI8kHAWw0/AACoQQAA2EEAABBBAADwQQAAwEEAAPBBAADwQQAAkEEAABBBJ97iQZ5glEGwYlm/AADwQQAAkEEAABBBAADwQQAAwEEAAPBBAABkQgAAEEEAABBBJAt9QqA33UA4VL4/AABkQgAAEEEAABBBAABAQgAAwEAAAPBBAAAcQgAAEEEAABBBmlciQvhvy0BQZwO/AAAcQgAAEEEAABBBAABAQgAAwEAAAPBBAABAQgAAAAAAABBBXrJPQrN6mcCEaJM/AABAQgAAAAAAABBBAABAQgAAwEAAAPBBAABAQAAA2EEAABBBL4mLwMSg

In [22]:
gauge_fcc_X1_samples = calculate_threshold('GaugeFloquetColourCodeX',[4,8,12], 1,3, np.linspace(0.0001,0.003, 7))
#gauge_hcc_X1_samples = calculate_threshold('GaugeHoneycombCodeX',[4,8,12], 1,3, np.linspace(0.0001,0.003, 7))
fcc_X1_samples = calculate_threshold('FloquetColourCodeX',[4,8,12], 1,3, np.linspace(0.0001,0.001, 2))
#hcc_X1_samples = calculate_threshold('HoneycombCodeX',[4,8,12], 1,3, np.linspace(0.0001,0.003, 7))


|████████████████████████████████████████✗︎ (!) 2307/2019 [114%] in 1.5s (1563.42/s)                                     1343.9/s, eta: 2s)  2064/2019 [102%] in 1s (1509.9/s, eta: 1s) 
|████████████████████████████████████████✗︎ (!) 2307/2019 [114%] in 1.4s (1661.93/s)                                     
|████████████████████████████████████████✗︎ (!) 2307/2019 [114%] in 1.4s (1697.37/s)                                     
|████████████████████████████████████████✗︎ (!) 2307/2019 [114%] in 1.1s (2014.50/s)                                     : 3s)  ▆█▆ 685/2019 [34%] in 1s (1156.6/s, eta: 2s) 
|████████████████████████████████████████✗︎ (!) 2307/2019 [114%] in 1.3s (1793.36/s)                                     : 4s)  ▇▅▃ 1864/2019 [92%] in 1s (1663.3/s, eta: 1s) 
|████████████████████████████████████████✗︎ (!) 2307/2019 [114%] in 2.1s (1095.40/s)                                     0.0/s, eta: -)  ▇▅▃ 1003/2019 [50%] in 1s (698.6/s, eta: 2s) 
|██████████████████████████████████████

Starting workers...
Finding work...
17 cases left:
    pymatching processes=1  ~core_mins_left=None    shots_left=1000000  errors_left=1000    {code=GaugeFloquetColourCodeX,distance=4,layers=12,p=0.0020333333333333336,r=3}
    pymatching processes=1  ~core_mins_left=None    shots_left=1000000  errors_left=1000    {code=GaugeFloquetColourCodeX,distance=4,layers=12,p=0.0025166666666666666,r=3}
    pymatching processes=1  ~core_mins_left=None    shots_left=1000000  errors_left=1000    {code=GaugeFloquetColourCodeX,distance=4,layers=12,p=0.003,r=3}
    pymatching processes=1  ~core_mins_left=None    shots_left=1000000  errors_left=1000    {code=GaugeFloquetColourCodeX,distance=8,layers=24,p=0.0001,r=3}
    pymatching processes=1  ~core_mins_left=None    shots_left=1000000  errors_left=1000    {code=GaugeFloquetColourCodeX,distance=8,layers=24,p=0.0005833333333333334,r=3}
    pymatching processes=1  ~core_mins_left=None    shots_left=1000000  errors_left=1000    {code=GaugeFloquetColourCode

|████████████████████████████████████████✗︎ (!) 579/507 [114%] in 0.6s (1013.63/s)                                       
|████████████████████████████████████████✗︎ (!) 579/507 [114%] in 0.5s (1257.30/s)                                       
|████████████████████████████████████████✗︎ (!) 1155/1011 [114%] in 3.9s (299.22/s)                                      3)             418/1011 [41%] in 2s (182.1/s, eta: 5s) 727/1011 [72%] in 3s (246.8/s, eta: 1s)  991/1011 [98%] in 3s (284.2/s, eta: 0s) (299.2/s, eta: -)                                 
|████████████████████████████████████████✗︎ (!) 1155/1011 [114%] in 3.5s (332.64/s)                                      )                                   352/1011 [35%] in 2s (197.8/s, eta: 5s)  ▂▂▄ 523/1011 [52%] in 2s (245.0/s, eta: 3s)  ▁▃▅ 581/1011 [57%] in 2s (257.7/s, eta: 2s)  643/1011 [64%] in 2s (270.1/s, eta: 2s) ▅▇▇ 807/1011 [80%] in 3s (298.3/s, eta: 1s) ▇▇▅ 891/1011 [88%] in 3s (310.0/s, eta: 1s)  █▆▄ 942/1011 [93%] in 3s (316.5

Starting workers...
Finding work...
5 cases left:
    pymatching processes=1  ~core_mins_left=None    shots_left=1000000  errors_left=1000    {code=FloquetColourCodeX,distance=4,layers=12,p=0.0001,r=3}
    pymatching processes=1  ~core_mins_left=None    shots_left=1000000  errors_left=1000    {code=FloquetColourCodeX,distance=8,layers=24,p=0.0001,r=3}
    pymatching processes=1  ~core_mins_left=None    shots_left=1000000  errors_left=1000    {code=FloquetColourCodeX,distance=8,layers=24,p=0.001,r=3}
    pymatching processes=1  ~core_mins_left=None    shots_left=1000000  errors_left=1000    {code=FloquetColourCodeX,distance=12,layers=36,p=0.0001,r=3}
    pymatching processes=1  ~core_mins_left=None    shots_left=1000000  errors_left=1000    {code=FloquetColourCodeX,distance=12,layers=36,p=0.001,r=3}
5 cases left:
    pymatching processes=6  ~core_mins_left=1       shots_left=924000   errors_left=949     {code=FloquetColourCodeX,distance=4,layers=12,p=0.0001,r=3}
    pymatching processes

In [ ]:
gauge_fcc_X1_samples = calculate_threshold('GaugeFloquetColourCodeX',[16], 1,3, np.linspace(0.001,0.003, 5))

In [4]:
gauge_hcc_X1_samples = calculate_threshold('GaugeHoneycombCodeX',[16], 1,3, np.linspace(0.001,0.003, 5))

|████████████████████████████████████████✗︎ (!) 4611/4035 [114%] in 1:59.1 (38.70/s)                                     n 1s (0.0/s, eta: -)  ▄▂▂ 0/4035 [0%] in 3s (0.0/s, eta: -) ▅▃▁ 0/4035 [0%] in 5s (0.0/s, eta: -) ▂▂▄ 0/4035 [0%] in 5s (0.0/s, eta: -) (0.0/s, eta: -)  ▂▄▆ 0/4035 [0%] in 5s (0.0/s, eta: -) (0.0/s, eta: -) (0.0/s, eta: -) ▂▄▆ 0/4035 [0%] in 14s (0.0/s, eta: -)  ▆█▆ 0/4035 [0%] in 15s (0.0/s, eta: -) 0/4035 [0%] in 18s (0.0/s, eta: -) (0.0/s, eta: -) 0/4035 [0%] in 21s (0.0/s, eta: -)  ▄▆█ 0/4035 [0%] in 26s (0.0/s, eta: -)  ▆█▆ 0/4035 [0%] in 28s (0.0/s, eta: -)  0/4035 [0%] in 29s (0.0/s, eta: -) in 29s (0.0/s, eta: -) (0.0/s, eta: -) (0.0/s, eta: -) in 36s (0.0/s, eta: -)  0/4035 [0%] in 37s (0.0/s, eta: -) (0.0/s, eta: -) 0/4035 [0%] in 37s (0.0/s, eta: -) (0.0/s, eta: -) (0.0/s, eta: -) 0/4035 [0%] in 49s (0.0/s, eta: -) 0/4035 [0%] in 49s (0.0/s, eta: -) in 53s (0.0/s, eta: -) ▇▅▃ 0/4035 [0%] in 53s (0.0/s, eta: -) (0.0/s, eta: -)  0/4035 [0%] in 57s (0.0/s, et

Starting workers...
Finding work...
5 cases left:
    pymatching processes=1  ~core_mins_left=None    shots_left=100000   errors_left=1000    {code=GaugeHoneycombCodeX,distance=16,layers=48,p=0.001,r=3}
    pymatching processes=1  ~core_mins_left=None    shots_left=100000   errors_left=1000    {code=GaugeHoneycombCodeX,distance=16,layers=48,p=0.0015,r=3}
    pymatching processes=1  ~core_mins_left=None    shots_left=100000   errors_left=1000    {code=GaugeHoneycombCodeX,distance=16,layers=48,p=0.002,r=3}
5 cases left:
    pymatching processes=1  ~core_mins_left=90      shots_left=99900    errors_left=1000    {code=GaugeHoneycombCodeX,distance=16,layers=48,p=0.001,r=3}
    pymatching processes=1  ~core_mins_left=None    shots_left=100000   errors_left=1000    {code=GaugeHoneycombCodeX,distance=16,layers=48,p=0.0015,r=3}
    pymatching processes=1  ~core_mins_left=None    shots_left=100000   errors_left=1000    {code=GaugeHoneycombCodeX,distance=16,layers=48,p=0.002,r=3}
5 cases left:
  

In [ ]:
fcc_X1_samples = calculate_threshold('FloquetColourCodeX',[16], 1,3, np.linspace(0.001,0.003, 5))

In [ ]:
hcc_X1_samples = calculate_threshold('HoneycombCodeX',[16], 1,3, np.linspace(0.0001,0.003, 7))

In [ ]:

def dem_histogram(dem):
    weight_dict = dict()
    total_errors = 0
    for dem_object in dem:
        if dem_object.type == "error":
            detector_ids = set()
            observable_ids = set()
            if len(dem_object.targets_copy()) in weight_dict:
                weight_dict[len(dem_object.targets_copy())] += 1
            else:
                weight_dict[len(dem_object.targets_copy())] = 1
            total_errors += 1
            #for target in dem_object.targets_copy():
            #    print('+1')
    for key in weight_dict:
        weight_dict[key] = weight_dict[key]/total_errors
    print(weight_dict, total_errors)


dem_histogram(circuit.detector_error_model(decompose_errors=True, approximate_disjoint_errors=True))
dem_histogram(circuit2.detector_error_model(decompose_errors=True, approximate_disjoint_errors=True))


{2: 0.3142857142857143, 3: 0.05510204081632653, 5: 0.30612244897959184, 4: 0.20612244897959184, 6: 0.08979591836734693, 7: 0.004081632653061225, 1: 0.024489795918367346} 5880
{2: 0.4859335038363171, 3: 0.023017902813299233, 4: 0.4629156010230179, 5: 0.020460358056265986, 1: 0.0076726342710997444} 4692


In [11]:
gauge_fcc_X1_samples = calculate_threshold('GaugeFloquetColourCodeX',[4], 1,1,np.linspace(0.001,0.003, 5))

|████████████████████████████████████████✗︎ (!) 771/675 [114%] in 0.8s (983.12/s)                                        s, eta: 0s) 
|████████████████████████████████████████✗︎ (!) 771/675 [114%] in 1.2s (618.14/s)                                         ▇▅▃ 661/675 [98%] in 1s (576.4/s, eta: 0s) 
|████████████████████████████████████████✗︎ (!) 771/675 [114%] in 1.1s (685.33/s)                                                           
|████████████████████████████████████████✗︎ (!) 771/675 [114%] in 1.2s (668.66/s)                                        
|████████████████████████████████████████✗︎ (!) 771/675 [114%] in 1.0s (783.18/s)                                        


Starting workers...
Finding work...
5 cases left:
    beliefmatching processes=1  ~core_mins_left=None    shots_left=5000     errors_left=500     {code=GaugeFloquetColourCodeX,distance=4,layers=4,p=0.001}
        pymatching processes=1  ~core_mins_left=None    shots_left=5000     errors_left=500     {code=GaugeFloquetColourCodeX,distance=4,layers=4,p=0.001}
    beliefmatching processes=1  ~core_mins_left=None    shots_left=5000     errors_left=500     {code=GaugeFloquetColourCodeX,distance=4,layers=4,p=0.0015}
        pymatching processes=1  ~core_mins_left=None    shots_left=5000     errors_left=500     {code=GaugeFloquetColourCodeX,distance=4,layers=4,p=0.0015}
    beliefmatching processes=2  ~core_mins_left=3       shots_left=4500     errors_left=382     {code=GaugeFloquetColourCodeX,distance=4,layers=4,p=0.002}
4 cases left:
    beliefmatching processes=1  ~core_mins_left=None    shots_left=5000     errors_left=500     {code=GaugeFloquetColourCodeX,distance=4,layers=4,p=0.001}
    

In [37]:
#hcc_X0_samples = calculate_threshold('HoneycombCodeX',[4,8,12],0,4,np.linspace(0.001,0.003,5))
#hcc_X1_samples = calculate_threshold('HoneycombCodeX',[4,8,12],1,1,np.logspace(-2.2,-1.9,10))

|████████████████████████████████████████✗︎ (!) 387/339 [114%] in 0.4s (872.64/s)                                        
|████████████████████████████████████████✗︎ (!) 387/339 [114%] in 0.3s (1283.02/s)                                       
|████████████████████████████████████████✗︎ (!) 387/339 [114%] in 0.3s (1334.71/s)                                       
|████████████████████████████████████████✗︎ (!) 387/339 [114%] in 0.3s (1381.70/s)                                       
|████████████████████████████████████████✗︎ (!) 387/339 [114%] in 0.3s (1364.29/s)                                       
|████████████████████████████████████████✗︎ (!) 771/675 [114%] in 2.2s (343.30/s)                                        █▆▄ 238/675 [35%] in 1s (228.3/s, eta: 3s)  536/675 [79%] in 2s (318.4/s, eta: 1s)  ▁▃▅ 591/675 [88%] in 2s (327.3/s, eta: 0s) 
|████████████████████████████████████████✗︎ (!) 771/675 [114%] in 2.3s (331.76/s)                                        /675 [50%] in 1s (24

Starting workers...
Finding work...
15 cases left:
        pymatching processes=1  ~core_mins_left=None    shots_left=5000     errors_left=500     {code=HoneycombCodeX,distance=4,layers=16,p=0.001}
    beliefmatching processes=1  ~core_mins_left=None    shots_left=5000     errors_left=500     {code=HoneycombCodeX,distance=4,layers=16,p=0.001}
        pymatching processes=1  ~core_mins_left=None    shots_left=5000     errors_left=500     {code=HoneycombCodeX,distance=4,layers=16,p=0.0015}
    beliefmatching processes=1  ~core_mins_left=None    shots_left=5000     errors_left=500     {code=HoneycombCodeX,distance=4,layers=16,p=0.0015}
        pymatching processes=1  ~core_mins_left=None    shots_left=5000     errors_left=500     {code=HoneycombCodeX,distance=4,layers=16,p=0.002}
    beliefmatching processes=1  ~core_mins_left=None    shots_left=5000     errors_left=500     {code=HoneycombCodeX,distance=4,layers=16,p=0.002}
        pymatching processes=1  ~core_mins_left=None    shots_lef